In [31]:
import pandas as pd
import numpy as np
from surprise import SVD, accuracy, SVDpp
from surprise import Dataset, Reader
import matplotlib.pyplot as plt
import math
from collections import defaultdict

In [32]:
train_data = pd.read_csv('train.csv')
valid_data = pd.read_csv('valid.csv')
test_data = pd.read_csv('test.csv')

In [33]:
print(len(train_data), len(valid_data), len(test_data))

905894 22647 203826


In [34]:
train_data

,user,item,rating
0,113920,108234,0
1,40482,15627,4
2,59345,17898,3
3,73568,143053,2
4,5015,20171,5
...,...,...,...
905889,39575,71024,5
905890,2210,12029,5
905891,2509,35326,5
905892,39523,58287,5


In [28]:
# let's only use 1/10 of the train data
# train_data = train_data.sample(frac=0.1, random_state=42)

In [35]:
def cossim(s1, s2):
  return len(s1.intersection(s2))/(math.sqrt(len(s1))*math.sqrt(len(s2)))

In [37]:
usersPerItem = defaultdict(set)
itemsPerUser = defaultdict(set)
itemSet = set([])
ratingDict = {}
meanRating = 0

for j in range(len(train_data)):
  u,i,r = train_data['user'][j], train_data['item'][j], train_data['rating'][j]
  if i not in itemSet:
        itemSet.add(i)
  itemsPerUser[u].add(i)
  usersPerItem[i].add(u)
  ratingDict[u,i] = r
  meanRating += r

meanRating/=len(train_data)
print(meanRating)

4.41098848209614


In [39]:
test_rating = {}
mse = 0

for j in range(len(test_data)):
    u,i,r = test_data['user'][j],test_data['item'][j],test_data['rating'][j]
    numerator = 0
    denominator = 0
    pred = meanRating

    if len(itemsPerUser[u]) != 0:
        for user in usersPerItem[i]:
            if user != u:
                sim = cossim(itemsPerUser[user],itemsPerUser[u])
                numerator += sim * ratingDict[user,i]
                denominator += sim
        
        pred = meanRating
        if denominator != 0 :
            pred = numerator/denominator

    test_rating[u,i] = (r,pred)
    mse+=(r - pred)**2

mse/=len(test_data)
print(mse) 

1.7178062574030326
